# Model Predict

In [1]:
import time
import six
import sys

import cifar_input
import numpy as np
import resnet_model
import tensorflow as tf

import os
import wget
import pandas as pd
from pathlib import Path


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
def get_file_name(img_url, f_name):
    try:
        file_name = wget.download(img_url, out= f_name)
    except Exception as e:
        print(e)
    return f_name

In [3]:
def get_image_tensor(f_name):
    image_size = 128
    batch_size = 1
    num_classes = 33
    
    filename_queue = tf.train.string_input_producer(tf.gfile.Glob(f_name))
    
    image_reader = tf.WholeFileReader()

    file_path, image_file = image_reader.read(filename_queue)
    
    
    image = tf.image.decode_jpeg(image_file, channels=3)
        
    longer = tf.reduce_max(tf.shape(image))
    image = tf.image.resize_image_with_crop_or_pad(image, longer, longer)
    image = tf.image.resize_images(image, [128, 128], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.image.resize_image_with_crop_or_pad(
            image, image_size, image_size)
    
    temp_image = image
    
    image = tf.image.per_image_standardization(image)
    label = tf.Variable([1])
    
    indices = tf.reshape(tf.range(0, batch_size, 1), [batch_size, 1])
    
    image = tf.expand_dims(image, 0)
    label = tf.expand_dims(label, 0)
    
    label = tf.sparse_to_dense(
        tf.concat(values=[indices, label], axis=1),
        [batch_size, num_classes], 1.0, 0.0)
    
    return image, label, temp_image

In [4]:
import joblib

cate_dict = joblib.load('./fasion_data/filtered_cate_mapping_dict_08_04')

In [5]:
cate_idx_dict = {}

for key, value in cate_dict.items():
    cate_idx_dict[value] = key

In [6]:
import os
from PIL import Image
import time


with tf.Session() as sess:
    
    batch_size = 1
    num_classes = 33

    hps = resnet_model.HParams(batch_size=batch_size,
                                   num_classes=num_classes,
                                   min_lrn_rate=0.0001,
                                   lrn_rate=0.1,
                                   num_residual_units=5,
                                   use_bottleneck=False,
                                   weight_decay_rate=0.0002,
                                   relu_leakiness=0.1,
                                   optimizer='mom')
    images = tf.placeholder(tf.float32, shape=(1, 128, 128, 3))
    labels = tf.placeholder(tf.int32, shape=(1, 33))
    model = resnet_model.ResNet(hps, images, labels, 'test')
    model.build_graph()
    
    saver = tf.train.Saver()
    log_root = 'amazon_log_08_04/'

    try:
        ckpt_state = tf.train.get_checkpoint_state(log_root)
    except tf.errors.OutOfRangeError as e:
        tf.logging.error('Cannot restore checkpoint: %s', e)

    timestr = time.strftime("%Y%m%d_%H%M%S")
    img_url = 'http://shop1.phinf.naver.net/20170706_19/realrealiz_1499282886420r6Byg_JPEG/22590079963655237_-573053896.jpg?type=m450'
    
    image, label, temp_image = get_image_tensor(get_file_name(img_url, timestr))
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess, os.getcwd() + '/' + ckpt_state.model_checkpoint_path)
    
    tf.train.start_queue_runners()
    
    img_eval = image.eval()
    label_eval = label.eval()
    
    
#     Image.fromarray(temp_image.eval(), 'RGB').save('bbb.jpeg')
    feed_dict = {images: img_eval, labels: label_eval}
    
    predictions, labels, m_images = sess.run([model.predictions, model.labels, model._images], feed_dict)
    predictions = np.argmax(predictions, axis=1)

    
    print(predictions)
    print(cate_idx_dict[predictions[0]])

INFO:tensorflow:Restoring parameters from /root/jude/project/fdm/amazon_log_08_04/model.ckpt-77403
[31]
Clothing, Shoes & Jewelry||Women||Clothing||Swimsuits & Cover Ups
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/RandomShuffle)]]


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/training/queue_runner_impl.py", line 238, in _run
    enqueue_callable()
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1063, in _single_operation_run
    target_list_as_strings, status, None)
  File "/usr/lib/python2.7/contextlib.py", line 24, in __exit__
    self.gen.next()
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/framework/errors_impl.py", line 466, in raise_exception_on_not_ok_status
    pywrap_tensorflow.TF_GetCode(status))
CancelledError: Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job

### 

In [11]:
cate_idx_dict[34]

'Clothing, Shoes & Jewelry||Boys||Clothing||Sleepwear & Robes'